<a href="https://colab.research.google.com/github/sfluhar/ChatGPT-and-Historical-Newspapers/blob/main/ChatGPT_and_Historical_Newspapers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
from tqdm import tqdm
import openai
import pandas as pd
import requests
import textwrap

In [ ]:
openai.api_key = '<enter your API key here>'
wrapper = textwrap.TextWrapper(width=150)

In [ ]:
api_search_url = 'https://chroniclingamerica.loc.gov/search/pages/results/'
topic = 'Wetzel County'
request_num = 20
num_pages_of_20_search_results = 11
first_or_all_pages = "all"
start_year = 1873
end_year = 1873

In [ ]:
chronology = "Analyze these documents in terms of chronology."
continuity_and_change = "Analyze these documents for specific periods of continuity and change over time."
perspective = "Analyze the points of view from which these documents were created."
corroboration = "Analyze how these documents corroborate each other."
contradiction = "Analyze how these documents contradict each other."
context = "Provide historical context for these documents."
motive = "Analyze the purposes for which these documents were made."
bias = "Analyze the bias of these documents."
contingency = "Analyze historical contingency in these documents."
multiple_causality = "Analyze multiple causality in these documents."
implicit = "Analyze the implicit meaning of these documents."
significance = "Analyze the historical significance of these documents."
empathy = "Use historical empathy to analyze these documents."
imagination = "Use historical imagination to analyze these documents."
synthesis = "Provide historical synthesis for these documents."
reliability = "Evaluate the reliability of these documents."
contestability = "Evaluate the contestability of these documents."
social_processes = "Analyze social processes in these documents."
social_theory = "Use social theory to analyze these documents."
sociological_theories = "Use sociological theories to analyze these documents."

thinking_requests = [chronology, continuity_and_change, perspective, corroboration, contradiction, context, motive, bias, contingency, multiple_causality, implicit,
                     significance, empathy, imagination, synthesis, reliability, contestability, social_processes, social_theory, sociological_theories]

In [ ]:
name_variable = {"Analyze these documents in terms of chronology.": "chronology",
                 "Analyze these documents for specific periods of continuity and change over time.": "continuity_and_change",
                 "Analyze the points of view from which these documents were created.": "perspective",
                 "Analyze how these documents corroborate each other.": "corroboration",
                 "Analyze how these documents contradict each other.": "contradiction",
                 "Provide historical context for these documents.": "context",
                 "Analyze the purposes for which these documents were made.": "motive",
                 "Analyze the bias of these documents.": "bias",
                 "Analyze historical contingency in these documents.": "contingency",
                 "Analyze multiple causality in these documents.": "multiple_causality",
                 "Analyze the implicit meaning of these documents.": "implicit",
                 "Analyze the historical significance of these documents.": "significance",
                 "Use historical empathy to analyze these documents.": "empathy",
                 "Use historical imagination to analyze these documents.": "imagination",
                 "Provide historical synthesis for these documents.": "synthesis",
                 "Evaluate the reliability of these documents.": "reliability",
                 "Evaluate the contestability of these documents.": "contestability",
                 "Analyze social processes in these documents.": "social_processes",
                 "Use social theory to analyze these documents.": "social_theory",
                 "Use sociological theories to analyze these documents.": "sociological_theories"}

In [ ]:
responses_df = pd.DataFrame(columns=['title','pub_place','date','ocr'])

In [ ]:
for year in tqdm(range(start_year,end_year+1)):
  params = { 'proxtext': topic }
  params['format'] = 'json'
  params['rows'] = request_num
  params['dateFilterType'] = 'yearRange'
  params['sort'] = 'relevance'
  params['date1'] = year
  params['date2'] = year
  if first_or_all_pages == "first":
    params['sequence'] = 1
  for page in range(num_pages_of_20_search_results):
    params['page'] = page + 1
    try:
      response = requests.get(api_search_url, params=params)
      data = response.json()
      pages_returned = len(data['items'])
      for item in range(pages_returned):
        title = data['items'][item]['title']
        pub_place = data['items'][item]['place_of_publication']
        date = data['items'][item]['date']
        ocr =  data['items'][item]['ocr_eng'].replace('\n',' ')
        responses_df = pd.concat([responses_df, pd.DataFrame.from_records([{ 'title': title, 'pub_place': pub_place, 'date': date, 'ocr': ocr}])], ignore_index=True)
    except:
      continue

100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


In [ ]:
responses_df = responses_df.drop_duplicates()

In [ ]:
responses_df.shape

(131, 4)

In [ ]:
responses_df.head()

,title,pub_place,date,ocr
0,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,"DAILY, TRI- WEEKLY, WEEKLY. LEW IS SAKKK A tC...."
1,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730613,ateiwliufl tHLEsnm ? cora. Our letter from W.t...
2,Spirit of Jefferson. [volume],"Charles Town, Va. [W. Va.]",18730624,"t-'&arlestoirn, Jefferson County, West Va. TPK..."
3,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730701,/? STEAMBOATS. For Cincinnati. . fn* * THE FIN...
4,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731018,FOR SALE. fHiuable t orI Property For NaJe. mo...


In [ ]:
print('OCR data quality sample:')
word_list = wrapper.wrap(text=responses_df.at[0,'ocr'][-1000:])
for element in word_list:
  print(element)

OCR data quality sample:
' i> m?d<? and timtM try J. M p*~, an<I w.f' to rnr, ri i't-'iDeloUr < 'li ' l <S r?ror?!?-'l In ho, 1, folio It*'. Id ftrvsot fheCerk < f tn?*
iv.urtjr <Vt?' '? cbto eocnly, Weal ?!? !: la. I 0 Saturday, Octobar 11th, 1873, At the front door of Ohio cronty ' ' Houw. proieed to Mil. beflonlnC
" o'clock p. m in? north half of I^H . '?n the ?iaat aide of Market ?treet, W " ' w?t v?., with 1'a ?ppart?D? ice*. lnsr of a large Three itory brick
d w*Vi hntiM>.and aerukUtwoatorydweJlrgt<~"' Bald I.o-.'vb will rent for ere.* WO- f-<r , TERMS or HAi. ?One ha f ??*& the residue In one and two
year*. ter?*t, t>*e deterred peymenta to be ?**?j / to the aatia'acUon of tb? tru*tf?, ? retained aa farther aecarlfy. .M Title believed to he good
beyond ?i"**1';*' !<ut aelllng aa troatee, i w:il convey C" ' kuoh title aa la In me. iiecta forov^ltCOper,^ aepts-t ' ' Til*'*4 GEO. A- WICKHAM,
Auctonter. The afv.re ?a!c fa roMponed tili/atnz^1/ ietobar 25. Iii7t at 10 o aloy a"^Ljbd 

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(
  model=model,
  messages=messages,
  temperature=0,)
  return response.choices[0].message["content"]

In [ ]:
completion_table = pd.DataFrame(columns=['title','pub_place','date','paragraph','keywords'])

In [ ]:
for item in tqdm(range(len(responses_df))):
  try:
    ocr =  responses_df.at[item, 'ocr']
    chunks = round((len(ocr) + 3899) // 3900, 0)
  except:
    continue
  for chunk in range(chunks):
    try:
      title = responses_df.at[item, 'title']
      pub_place = responses_df.at[item, 'pub_place']
      date = responses_df.at[item, 'date']
      body = ocr[chunk*3900:(chunk+1)*3900]
      chat_input = str(title + " " + pub_place + " " + date + " " + body)
      chat_response = get_completion("Here is a historical document: " + chat_input + " | " + "In one paragraph, write a narrative of the people, \
                                    organizations, places, dates, events, and things in this historical document that have a direct connection to " + topic)
      if "direct" not in chat_response and "connection" not in chat_response and "no mention" not in chat_response and "not enough" not in chat_response:
        main_topic = get_completion("Here is a historical document: " + chat_response
                                    + " List one to three single-word subjects for this document. \
                                    Make each of these single-word subjects lower-case and separate them by commas.")
        completion_table = pd.concat([completion_table,
                                      pd.DataFrame.from_records([{ 'title': title, 'pub_place': pub_place, 'date': date,
                                                              'paragraph': chat_response, 'keywords': main_topic}])], ignore_index=True)
    except:
      continue

100%|██████████| 131/131 [2:28:52<00:00, 68.19s/it] 


In [ ]:
completion_table.shape

(108, 5)

In [ ]:
completion_table.head()

,title,pub_place,date,paragraph,keywords
0,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,"The historical document, The Wheeling daily re...","bullying, politics, press"
1,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,"Unfortunately, the historical document provide...","unknown, unrelated, irrelevant"
2,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,The historical document is an advertisement fr...,"watches, land, advertisement"
3,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730613,The historical document reports on the lynchin...,"lynching, justice, vigilantism"
4,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730613,"In the historical document, ""The Wheeling dail...","lynching, crime, violence"


In [ ]:
total_char = sum([len(line) for line in completion_table.paragraph.values])
print(total_char)

56885


In [ ]:
chunks = round((total_char + 7999) // 8000, 0)
print(chunks)

8


In [ ]:
if completion_table.shape[0] < 7:
  true_k = max(completion_table.shape[0],chunks)
else:
  true_k = max(7,chunks)
print("There are",true_k,"clusters")

There are 8 clusters


In [ ]:
stop_words = get_stop_words('en')
text = completion_table.paragraph.values
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(text)
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
model.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


KMeans(max_iter=200, n_init=10)

In [ ]:
labels = model.labels_
clusters = pd.DataFrame(list(zip(text,labels)), columns=['title','cluster'])
completion_clusters = pd.merge(completion_table, clusters.cluster, left_index=True, right_index=True)

In [ ]:
completion_clusters.head()

,title,pub_place,date,paragraph,keywords,cluster
0,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,"The historical document, The Wheeling daily re...","bullying, politics, press",5
1,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,"Unfortunately, the historical document provide...","unknown, unrelated, irrelevant",1
2,The Wheeling daily register. [volume],"Wheeling, W. Va.",18731016,The historical document is an advertisement fr...,"watches, land, advertisement",4
3,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730613,The historical document reports on the lynchin...,"lynching, justice, vigilantism",2
4,The Wheeling daily register. [volume],"Wheeling, W. Va.",18730613,"In the historical document, ""The Wheeling dail...","lynching, crime, violence",1


In [ ]:
completion_clusters.to_csv("articles_by_cluster.csv", index=False)
files.download("articles_by_cluster.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cluster_keywords_df = pd.DataFrame(columns=['cluster','keywords'])

In [ ]:
for cluster_num in tqdm(sorted(completion_clusters.cluster.unique())):
  cluster_slice = completion_clusters[completion_clusters.cluster==cluster_num].reset_index(drop=True)
  key_list = []
  for item in range(len(cluster_slice)):
    keywords = cluster_slice.at[item, 'keywords']
    key_split = keywords.split(", ")
    for key in key_split:
      key_list.append(str(key))
  key_dedup = list(set(key_list))
  cluster_keywords = ", ".join(key_dedup)
  cluster_keywords_df = pd.concat([cluster_keywords_df,
                                   pd.DataFrame.from_records([{'cluster': cluster_num, 'keywords': cluster_keywords}])], ignore_index=True)
cluster_keywords_df.to_csv("cluster_keywords.csv", index=False)
files.download("cluster_keywords.csv")

100%|██████████| 8/8 [00:00<00:00, 89.36it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def thinking_skill(completion_clusters, thinking_skills_df, request):
  completion_clusters.sort_values('date')
  for cluster_num in sorted(completion_clusters.cluster.unique()):
    cluster_slice = completion_clusters[completion_clusters.cluster==cluster_num].reset_index(drop=True)
    chat_string = ""
    for item in range(len(cluster_slice)):
      title = cluster_slice.at[item, 'title']
      pub_place = cluster_slice.at[item, 'pub_place']
      date = cluster_slice.at[item, 'date']
      paragraph = cluster_slice.at[item, 'paragraph']
      if item < len(cluster_slice) - 1:
        chat_string += date + title + pub_place + ":" + paragraph + " | "
      else:
        chat_string += date + title + pub_place + ":" + paragraph
    chunk = chat_string[:8000]
    try:
      response = get_completion("Here are some historical documents: " + chunk + " | " + request)
      thinking_skills_df = pd.concat([thinking_skills_df,
                                      pd.DataFrame.from_records([{'cluster': cluster_num, 'request': request, 'response': response}])], ignore_index=True)
      thinking_skills_df.to_csv(name_variable[request]+".csv", index=False)
    except:
      continue

In [ ]:
for request in tqdm(thinking_requests):
  thinking_skills_df = pd.DataFrame(columns=['cluster','request','response'])
  thinking_skill(completion_clusters, thinking_skills_df, request)
  files.download(name_variable[request]+".csv")

  0%|          | 0/20 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  5%|▌         | 1/20 [00:51<16:20, 51.60s/it]

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

 95%|█████████▌| 19/20 [29:48<01:16, 76.70s/it]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

100%|██████████| 20/20 [31:15<00:00, 93.78s/it]
